In [1]:
# from trade_rl.meta.data_processors.alpaca import Alpaca
import alpaca_trade_api as tradeapi
import pandas as pd

In [2]:
keys = {}
keys["API_KEY"] = "PK2IGPKCURWCF4CEYZ1G"
keys["API_SECRET"] = "OUZUEJrGtxY6ZCcPnCDsbsq8bfHUehMQm6eGEep4"
keys["API_BASE_URL"] = 'https://paper-api.alpaca.markets'
keys["data_url"] = 'wss://data.alpaca.markets'
api = tradeapi.REST(
                    keys["API_KEY"],
                    keys["API_SECRET"],
                    keys["API_BASE_URL"],
                    "v2",
                )

In [3]:
time_interval = "1Min"
start_date = pd.Timestamp('2022-09-01', tz="Europe/Paris" ).date().isoformat()
end_date = pd.Timestamp('2022-09-03', tz="Europe/Paris" ).date().isoformat()

In [199]:
barset = api.get_crypto_bars(
                    ["ETHUSD", "BTCUSD"],
                    time_interval,
                    exchanges='CBSE',
                ).df.groupby("symbol").tail(100)

In [202]:
len(barset)

200

In [91]:
barset = barset.reset_index()
barset["time"] = barset["timestamp"].apply(
            lambda x: x.strftime("%Y-%m-%d %H:%M:%S")
        )

In [92]:
barset = barset.rename(columns={"symbol":"tic"})

In [100]:
import stockstats
from typing import List
def test_stock_stats(dataframe: pd.DataFrame, tech_indicator_list:List[str]):
    stock = stockstats.StockDataFrame.retype(dataframe)
    unique_ticker = stock.tic.unique()
    for indicator in tech_indicator_list:
        print("indicator: ", indicator)
        indicator_df = pd.DataFrame()
        for i in range(len(unique_ticker)):
            try:
                temp_indicator = stock[stock.tic ==
                                        unique_ticker[i]][indicator]
                temp_indicator = pd.DataFrame(temp_indicator)
                temp_indicator["tic"] = unique_ticker[i]
                temp_indicator["time"] = dataframe[
                    dataframe.tic == unique_ticker[i]
                ]["time"].to_list()
                indicator_df = pd.concat(
                    [indicator_df, temp_indicator],
                    axis=0,
                    join="outer",
                    ignore_index=True,
                )
            except Exception as e:
                print(e)
        if not indicator_df.empty:
            dataframe = dataframe.merge(
                indicator_df[["tic", "time", indicator]],
                on=["tic", "time"],
                how="left",
            )

    return dataframe

tech_indicator_list = ['macd', 'rsi', 'cci', 'dx', 'rsi_30']
df = test_stock_stats(barset, tech_indicator_list)

indicator:  macd
indicator:  rsi
indicator:  cci
indicator:  dx
indicator:  rsi_30


In [101]:
df.backfill().head()

,timestamp,exchange,open,high,low,close,volume,trade_count,vwap,tic,time,macd,rsi,cci,dx,rsi_30
0,2022-09-01 05:00:00+00:00,CBSE,20054.10,20055.23,20044.63,20047.37,4.858587,252,20049.279334,BTCUSD,2022-09-01 05:00:00,0.000000,100.000000,66.666667,100.000000,100.000000
1,2022-09-01 05:01:00+00:00,CBSE,20048.74,20063.02,20047.58,20061.69,12.714989,243,20058.593336,BTCUSD,2022-09-01 05:01:00,0.321282,100.000000,66.666667,100.000000,100.000000
2,2022-09-01 05:02:00+00:00,CBSE,20061.31,20063.36,20044.57,20045.57,6.147744,312,20056.755726,BTCUSD,2022-09-01 05:02:00,-0.090503,45.202020,-28.551881,38.328097,46.199715
3,2022-09-01 05:03:00+00:00,CBSE,20045.16,20054.84,20042.44,20044.23,6.259407,307,20047.215038,BTCUSD,2022-09-01 05:03:00,-0.323953,43.088296,-86.631828,10.505265,44.156823
4,2022-09-01 05:04:00+00:00,CBSE,20044.75,20048.70,20036.21,20039.42,6.062276,298,20042.382135,BTCUSD,2022-09-01 05:04:00,-0.635388,36.491833,-129.174106,34.176373,37.928941


In [4]:
from trade_rl.meta.data_processors.alpaca_crypto import AlpacaCrypto
DP = AlpacaCrypto(data_source="alpaca", start_date='2022-09-01', end_date='2022-09-03', time_interval="1Min",
                 **keys)

In [5]:
df = DP.download_data(ticker_list= ["ETHUSD", "BTCUSD"])

In [8]:
DP.dataframe

,timestamp,exchange,open,high,low,close,volume,trade_count,vwap,tic,time
0,2022-09-01 05:00:00+00:00,CBSE,20054.10,20055.23,20044.63,20047.37,4.858587,252,20049.279334,BTCUSD,2022-09-01 05:00:00
1,2022-09-01 05:01:00+00:00,CBSE,20048.74,20063.02,20047.58,20061.69,12.714989,243,20058.593336,BTCUSD,2022-09-01 05:01:00
2,2022-09-01 05:02:00+00:00,CBSE,20061.31,20063.36,20044.57,20045.57,6.147744,312,20056.755726,BTCUSD,2022-09-01 05:02:00
3,2022-09-01 05:03:00+00:00,CBSE,20045.16,20054.84,20042.44,20044.23,6.259407,307,20047.215038,BTCUSD,2022-09-01 05:03:00
4,2022-09-01 05:04:00+00:00,CBSE,20044.75,20048.70,20036.21,20039.42,6.062276,298,20042.382135,BTCUSD,2022-09-01 05:04:00
...,...,...,...,...,...,...,...,...,...,...,...
8635,2022-09-04 04:55:00+00:00,CBSE,1554.71,1555.41,1554.19,1555.37,119.797993,195,1554.771144,ETHUSD,2022-09-04 04:55:00
8636,2022-09-04 04:56:00+00:00,CBSE,1555.17,1555.53,1554.47,1555.30,43.001448,105,1554.961914,ETHUSD,2022-09-04 04:56:00
8637,2022-09-04 04:57:00+00:00,CBSE,1555.32,1556.82,1555.03,1556.45,67.852859,128,1555.837022,ETHUSD,2022-09-04 04:57:00
8638,2022-09-04 04:58:00+00:00,CBSE,1556.59,1557.01,1555.71,1556.09,59.226152,141,1556.106839,ETHUSD,2022-09-04 04:58:00
